# Preprocessing code

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import transform, util
import nibabel as nib
import tqdm
from PIL import Image
from skimage.restoration import denoise_tv_chambolle
from skimage.measure import regionprops, label
import matplotlib as mpl
%matplotlib inline

def get_ROI(im,mask,res):
    # Give as an input an image (in the form of a dictionary with labels 'img'and 'mask) 
    # and outputs an ROI centred around the centre of mass of the heart. 
    # Second input is the resolutoin (mm/pixel) of the input image.
    
    # # split input into image and mask
    # im = image['img']                          
    # mask = image['mask']
    
    # find target size of ROI, based on pixel size (mm/pixel) and maximum expected size of heart (doi: 10.15171/jcvtr.2016.25)
    # nr = np.round(16/res[0]*10/2).astype(int) # number of rows/2
    # nc = np.round(16/res[1]*10/2).astype(int) # number of columns/2
    nr = int(128/2) # target size is 128 by 128 pixels
    nc = int(128/2)
    mask_single_class = np.copy(mask)
    # mask_single_class = np.expand_dims(mask_single_class, axis=0)
    # print(mask_single_class)
    # print(np.shape(mask_single_class))
    # print(np.where(mask_single_class!=0))
    # print(np.unique(mask_single_class))
    mask_single_class[mask_single_class!=0] = 1
    if len(np.unique(mask_single_class))==1:                                            # if there is no heart in the image, just use the central part of the image as an ROI
        labels = label(mask_single_class,background=1)                                  # find all connected components
        largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1    # largest connected components
        mask = largestCC*1                                                 # True --> 1, False --> 0     
        properties = regionprops(mask)                                     # Geometrical properties of image
        com = np.round(properties[0].centroid).astype(int)                 # Centre of mass = centre of image in this specific case
        com_r = com[0]                                                     # row index
        com_c = com[1]                                                     # column index
        ROI_mask = mask[(com_r-nr):(com_r+nr),(com_c-nc):(com_c+nc)]     # ROI mask --> traget size 128*128 pixels
        ROI_mask = 0*ROI_mask
        ROI = im[(com_r-nr):(com_r+nr),(com_c-nc):(com_c+nc)]            # ROI of grey scale image

    else:                                                                  # if heart is present
        # Find Largest connected component:
        labels = label(mask_single_class)                                  # find all connected components
        largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1    # largest connected components
        largestCC = largestCC*1                                            # True --> 1, False --> 0
        properties = regionprops(largestCC)                                # Geometrical properties of largest connected component
        com = np.round(properties[0].centroid).astype(int)                 # Centre of mass
        com_r = com[0]                                                     # row index
        com_c = com[1]                                                     # column index
        ROI_mask = mask[(com_r-nr):(com_r+nr),(com_c-nc):(com_c+nc)]
        # print(np.unique(ROI_mask))
        ROI = im[(com_r-nr):(com_r+nr),(com_c-nc):(com_c+nc)]    
    # plt.figure()
    # plt.imshow(np.squeeze(ROI),'gray')
    # overlay_mask = np.ma.masked_where(ROI_mask==0, ROI_mask)
    # plt.imshow(overlay_mask, 'Reds', alpha = 0.5, clim=[0,1], interpolation='nearest')
    # plt.scatter(x=nc, y=nr, c='g', s=40)
    # plt.figure()
    
    return ROI,ROI_mask


def normalize_img(img):
    # Warning for when dividing NaN value??
    norm_img = np.divide(img,np.max(img))
    return norm_img


def crop_pad_resize(image, nx, ny):
    x, y = image.shape

    # difference in nr of pixels (divide by 2 since we have 2 sides)
    x_s = (x - nx) // 2
    y_s = (y - ny) // 2
    x_c = (nx - x) // 2
    y_c = (ny - y) // 2

    if x > nx and y > ny:
        # if image is larger in both dimensions cut a slice
        slice_cropped = image[x_s:x_s + nx, y_s:y_s + ny]

    else:
        # if one dim is smaller fill that side up with 0's
        slice_cropped = np.zeros((nx, ny))

        if x <= nx and y > ny:
            # fill up x direction with 0's, cut in x direction
            slice_cropped[x_c:x_c + x, :] = image[:, y_s:y_s + ny]
        elif x > nx and y <= ny:
            # fill up y direction with 0's, cut in y direction
            slice_cropped[:, y_c:y_c + y] = image[x_s:x_s + nx, :]
        else:
            # if dimensions are as desired, keep the original slice
            slice_cropped[x_c:x_c + x, y_c:y_c + y] = image[:, :]

    return slice_cropped


def preprocess(input_folder, target_resolution, target_size, denoise=False, alphaTV=0.1):
    '''
    This function preprocesses ACDC data. It crops all images to the same size,
    transforms everything to the same resolution and normalizes the images.
    It automatically makes the folder where preprocessed data is written to,
    in the same format as the ACDC data is given. The images are in PNG-format.
    
    input_folder: the folder where raw ACDC data is located.
    target_resolution: desired resolution, should be a tuple with 2 items (x- and y-dimensions).
    target_size: desired size. Should be a tuple wiht 2 items (x- and y-dimensions).
    '''
    corrupted_files = []
    nx, ny = target_size
    data_folder = input_folder
    
    if denoise:
        foldername = 'preprocessed_ROI_TV005_v3'
    else:
        foldername = 'preprocessed_ROI_v3'
    
    if not os.path.exists(foldername):
        os.mkdir(foldername)
    else:
        print(foldername+' folder already exists. Continuing regardless.')
    
    # Loop over train and test folders
    for train_test in ['training', 'testing']:

        input_folder = os.path.join(data_folder, train_test)
        len_inp = len(input_folder)+1
        
        # Make train and test folders in preprocessed folder
        if not os.path.exists(os.path.join(foldername+'/', train_test)):
            os.mkdir(os.path.join(foldername+'/', train_test))
        else:
            print('T'+train_test[1:]+' folder already exists. Continuing regardless.')
        
        # Loop over patient folders
        for folder in os.listdir(input_folder):
            
            if folder != '.ipynb_checkpoints':  # Sometimes trouble with automatically made files

                folder_path = os.path.join(input_folder, folder)
                
                # Make patient folders in preprocessed folder
                if not os.path.exists(os.path.join(foldername+'/'+train_test, folder_path[len_inp:])):
                    os.mkdir(os.path.join(foldername+'/'+train_test, folder_path[len_inp:]))
                else:
                    print('Folder for '+folder_path[len_inp:]+' already exists. Continuing regardless.')
                
                if os.path.exists(foldername+'/'+train_test+'/'+folder_path[len_inp:]+'/.ipynb_checkpoints'):
                    os.rmdir(foldername+'/'+train_test+'/'+folder_path[len_inp:]+'/.ipynb_checkpoints')
                    
                lst = os.listdir(foldername+'/'+train_test+'/'+folder_path[len_inp:])
                
                if len(lst) == 0:  # Only create files if the designated folder is empty
                    
                    for file in glob.glob(os.path.join(folder_path, 'patient???_frame??.nii.gz')):

                        # Save information about patient
                        with open(os.path.join(folder_path, 'Info.cfg')) as f:
                            lines = f.readlines()

                        ED = int(lines[0].strip()[-2:])
                        ES = int(lines[1].strip()[-2:])

                        # Split file name
                        file_base = file.split('.nii.gz')[0]
                        file_mask = file_base + '_gt.nii.gz'

                        # Load data from .nii.gz files
                        img_nii = nib.load(file)
                        img_dat = img_nii.get_fdata()

                        mask_nii = nib.load(file_mask)
                        mask_dat = mask_nii.get_fdata().astype(int)
                        
                        # if(np.logical_or('patient038' in file , 'patient057' in file)):
                        #     print(np.unique(mask_dat))


                        img = img_nii.get_fdata()
                        mask = mask_nii.get_fdata()

                        pixel_size = img_nii.header.get_zooms()

                        # Make vector to make all images have the same resolution
                        scale_vector = [pixel_size[0] / target_resolution[0], pixel_size[1] / target_resolution[1]] 

                        for zz in tqdm.tqdm(range(img.shape[2])):

                            # Normalize, rescale and crop the image and  mask

                            slice_img = np.squeeze(img[:, :, zz])
                            slice_img = normalize_img(np.squeeze(img[:, :, zz]))
                            #slice_img = np.squeeze(img[:, :, zz])
                            img_rescaled = transform.rescale(slice_img,
                                                             scale_vector,
                                                             order=1,
                                                             preserve_range=True,
                                                             mode='constant')

                            slice_mask = np.squeeze(mask[:, :, zz])
                            # slice_mask = normalize_img(np.squeeze(mask[:, :, zz]))
                            mask_rescaled = np.around(transform.rescale(slice_mask,
                                                              scale_vector,
                                                              order=0,
                                                              preserve_range=True,
                                                              mode='constant'))
                            #print(np.unique(mask_rescaled))
                            img_cropped = crop_pad_resize(img_rescaled, nx, ny)
                            mask_cropped = crop_pad_resize(mask_rescaled, nx, ny)
                                                       
                            
                            # if(np.logical_or('patient038' in file , 'patient057' in file)):
                            #     #print(np.unique(mask_rescaled))
                            #     # print(file_base)
                            #     corrupted_files.append(file_base)
                            # else:
                            
                            # Find largest connected component:
                            img_cropped,mask_cropped=get_ROI(img_cropped,mask_cropped,target_resolution)
                                
                            if denoise:
                                img_cropped = denoise_tv_chambolle(img_cropped, eps=1e-6, weight=alphaTV, max_num_iter=1000)
                            
                            # if not (np.logical_or('patient038' in file , 'patient057' in file)):
                            # Save images in PNG format
                            if 'frame{:02}'.format(ED) in file:
                                img_loc = os.path.join(foldername+'/'+train_test, file[len_inp:-7]+'_slice{:01}_ED'.format(zz)+'.png')
                                img_fin = Image.fromarray(np.uint8(255*img_cropped),mode="L")
                                img_fin.save(img_loc, format='PNG')

                                mask_loc = os.path.join(foldername+'/'+train_test, file[len_inp:-7]+'_slice{:01}_ED_gt'.format(zz)+'.png')
                                mask_fin = Image.fromarray(np.uint8(mask_cropped), mode="L")
                                #print('uniquecrop'+ str(np.uint8(np.unique(mask_cropped))))
                                mask_fin.save(mask_loc, format='PNG')
                            else:
                                img_loc = os.path.join(foldername+'/'+train_test, file[len_inp:-7]+'_slice{:01}_ES'.format(zz)+'.png')
                                img_fin = Image.fromarray(np.uint8(255*img_cropped), mode="L")
                                img_fin.save(img_loc, format='PNG')

                                #print(np.shape(mask_cropped))
                                mask_loc = os.path.join(foldername+'/'+train_test, file[len_inp:-7]+'_slice{:01}_ES_gt'.format(zz)+'.png')
                                mask_fin = Image.fromarray(np.uint8(mask_cropped),mode="L")
                                mask_fin.save(mask_loc, format='PNG')
                else:
                    print('Folder for '+folder_path[len_inp:]+' is not empty. No files were written to this folder.')
        
    # print(corrupted_files)
    print("Preprocessing Finished")



#### Preprocess the data

In [9]:
target_resolution = (1.36719, 1.36719)
target_size = (256, 256)
data_path = '../ACDC/database/'

#preprocess(data_path, target_resolution, target_size)
preprocess(data_path, target_resolution, target_size, denoise=False)

100%|██████████| 10/10 [00:00<00:00, 42.56it/s]

Preprocessing Finished


#### Denoise the data

In [10]:
# preprocess(data_path, target_resolution, target_size, denoise=True, alphaTV=0.2)
print('done')
# alpha = 0.1 geeft wel prima maar nog veel details, 0.3 is misschien net hoog.

done


In [ ]:
# ik heb alpha =0.1 gekozen, target size veranderd naar 256*256 --> werkt beter icm onze kernal afmetingen en aantal lagen van netwerk, voor gecropte images gebruik ik een target size van 128*128

# Voor ROI alpha verlaagd naar 0.05 ---> toch naar 0.1